In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

## Dates

In [2]:
# Define path to files
# For MacOS
# PATH = "/Users/derekbanas/Documents/Tutorials/Python for Finance/Stocks/"
# For Windows
PATH = "C:/Users/CLeen/Documents/GitHub/Python4Finance/stock_list/"

# Start date defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2017-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2021
E_MONTH = 8
E_DAY = 19
E_DATE_STR = "2021-08-19"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

## Get Stock File Names in a List

In [3]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

# On MacOS Only
# tickers.remove('.DS_Store')
tickers.sort()
tickers

['A',
 'AA',
 'AAL',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAWW',
 'ABR',
 'ABT',
 'USEG',
 'USFD',
 'USLM',
 'USM',
 'USNA',
 'USPH',
 'USWS',
 'USX',
 'UTHR',
 'UTI',
 'UTL',
 'UTMD',
 'UUU',
 'UVE',
 'UVSP',
 'UVV',
 'V',
 'VAC',
 'VAL',
 'VALU',
 'VAPO',
 'VBFC',
 'VBIV',
 'VBTX',
 'VC',
 'VCEL',
 'VCRA',
 'VCTR',
 'VCYT',
 'VEC',
 'VECO',
 'VEEV',
 'VERI',
 'VERU',
 'VFC',
 'VG',
 'VGR',
 'VHC',
 'VIAV',
 'VICI',
 'VICR',
 'VIRC',
 'VIRT',
 'VIVO',
 'VKTX',
 'VLGEA',
 'VLO',
 'VLY',
 'VMC',
 'VMI',
 'VMW',
 'VNCE',
 'VNDA',
 'VNE',
 'VNO',
 'VNRX',
 'VNTR',
 'VOXX',
 'VOYA',
 'VPG',
 'VRA',
 'VRAY',
 'VREX',
 'VRNS',
 'VRNT',
 'VRRM',
 'VRS',
 'VRSK',
 'VRSN',
 'VRTS',
 'VRTV',
 'VRTX',
 'VSAT',
 'VSEC',
 'VSH',
 'VST',
 'VSTM',
 'VSTO',
 'VTNR',
 'VTR',
 'VTVT',
 'VUZI',
 'VVI',
 'VVV',
 'VXRT',
 'VYGR',
 'VZ',
 'W',
 'WAB',
 'WABC',
 'WAFD',
 'WAL',
 'WASH',
 'WAT',
 'WATT',
 'WBA',
 'WBS',
 'WBT',
 'WCC',
 'WD',
 'WDAY',
 'WDC',
 'WDFC',
 'WEC',
 'WELL',

## Function that Returns a Dataframe from a CSV

In [21]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(ticker):
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(PATH + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

## Function that Saves Dataframe to CSV

In [22]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

## Return 1st Valid Date in Dataframe

In [23]:
# Receives a start and end date and returns the 1st date in that range
def get_valid_dates(df, sdate, edate):
    
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
    
        # Get smallest date that matches
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
    
        # Add leading zeros to date
        # Split string on - and if a number is less than 2 add a zero in front
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
        print(date_leading, " ", date_ending)
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending

## Returns Return on Investment over Time

In [24]:
# Get ROI between 2 dates
def roi_between_dates(df, sdate, edate):
    
    try:
        # Gets the Adj Close price for 1st & last date 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi

## Get Mean Between Dates

In [25]:
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].mean()

## Get Standard Deviation Between Dates

In [26]:
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].std()

## Get Coefficient of Variation Between Dates

In [27]:
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd / mean

## Test Functions

In [31]:
# Get our 1st ticker
tickers[0]
print("Dataframe for Ticker", tickers[0])

# Get a dataframe for that ticker
stock_a = get_df_from_csv(tickers[0])
stock_a

# # Get the first and last valid date in the date range
print(get_valid_dates(stock_a, '2020-01-01', '2020-12-31'))
sdate, edate = get_valid_dates(stock_a, '2020-01-01', '2020-12-31')
sdate
edate

print("Adj Close Mean :", get_mean_between_dates(stock_a, sdate, edate))
print("Adj Close Standard Deviation :", get_sd_between_dates(stock_a, sdate, edate))
print("Adj Close Coefficient of Variation :", get_cov_between_dates(stock_a, sdate, edate))

stock_a = stock_a.set_index(['Date'])
stock_a

print("Return on Investment since 2020:", roi_between_dates(stock_a, sdate, edate))

Dataframe for Ticker A
2020-01-02   2020-12-31
('2020-01-02', '2020-12-31')
2020-01-02   2020-12-31
Adj Close Mean : 91.76716743953644
Adj Close Standard Deviation : 13.793250598387852
Adj Close Coefficient of Variation : 0.15030703227792175
Return on Investment since 2020: 0.38750008760499793


## COV & ROI for All Stocks Over Defined Period

In [29]:
def get_cov_ror(tickers, sdate, edate):
    # Define column names for dataframe
    col_names = ["Ticker", "COV", "ROI"]
    
    # Create dataframe with column names
    df = pd.DataFrame(columns = col_names)
    
    for ticker in tickers:
        print("Working on :", ticker)
        s_df = get_df_from_csv(ticker)
    
        sdate2, edate2 = get_valid_dates(s_df, sdate, edate)
    
        cov = get_cov_between_dates(s_df, sdate2, edate2)
    
        # Set date as index
        s_df = s_df.set_index(['Date'])
        roi = roi_between_dates(s_df, sdate2, edate2)

        # Add stock data to new dataframe row
        # len provides the length of the dataframe which is the next open index
        df.loc[len(df.index)] = [ticker, cov, roi]
    
    return df

In [33]:
# Remove CRC GRUB AAN ARNC
market_df = get_cov_ror(tickers, '2021-01-03', '2021-12-31')

Working on : A
2021-01-04   2021-08-19
Working on : AA
2021-01-04   2021-08-19
Working on : AAL
2021-01-04   2021-08-19
Working on : AAME
2021-01-04   2021-08-19
Working on : AAN
2021-01-04   2021-08-19
Working on : AAOI
2021-01-04   2021-08-19
Working on : AAON
2021-01-04   2021-08-19
Working on : AAP
2021-01-04   2021-08-19
Working on : AAPL
2021-01-04   2021-08-19
Working on : AAT
2021-01-04   2021-08-19
Working on : AAWW
2021-01-04   2021-08-19
Working on : ABR
2021-01-04   2021-08-19
Working on : ABT
2021-01-04   2021-08-19
Working on : USEG
2021-01-04   2021-08-19
Working on : USFD
2021-01-04   2021-08-19
Working on : USLM
2021-01-04   2021-08-19
Working on : USM
2021-01-04   2021-08-19
Working on : USNA
2021-01-04   2021-08-19
Working on : USPH
2021-01-04   2021-08-19
Working on : USWS
2021-01-04   2021-08-19
Working on : USX
2021-01-04   2021-08-19
Working on : UTHR
2021-01-04   2021-08-19
Working on : UTI
2021-01-04   2021-08-19
Working on : UTL
2021-01-04   2021-08-19
Working

In [34]:
# Get top 20 stocks ROI
market_df.sort_values(by=['ROI'], ascending=False).head(20)

,Ticker,COV,ROI
91,VTNR,0.850821,9.013193
83,VRTV,0.374937,2.782336
150,WOW,0.226875,0.955401
110,WBT,0.211010,0.847255
155,WRLD,0.142691,0.796949
19,USWS,0.344003,0.794872
3,AAME,0.204030,0.714514
66,VNE,0.167404,0.710403
90,VSTO,0.149893,0.655229
127,WHG,0.197154,0.633752


## Merge Multiple Stocks in One Dataframe by Column Name

In [35]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

## Create a Correlation Matrix using FAANGS

In [36]:
# Correlation tells us how closely 2 stocks returns move together
# Correlation is a standardized value lying between -1 and 1
# When this value is greater that .5 we say that these stocks are strongly correlated
# Of course each stocks price is perfectly correlated with itself

# We focus on the correlation of returns because investors care about returns 

# Merge all stock price data into 1 dataframe
faang_list = ["FB", "AMZN", "AAPL", "NFLX", "GOOG"]
mult_df = merge_df_by_column_name('daily_return',  '2021-1-1', '2021-12-31', *faang_list)
mult_df

# Generate a Correlation Matrix
mult_df.corr()

# We can look at the correlation between Netflix and the others
mult_df.corr()['NFLX']

# We can plot this in a bar chart
mult_df.corr()['NFLX'].plot(kind='bar')

File Doesn't Exist


TypeError: 'NoneType' object is not subscriptable

## Get the Variance of a Stock

In [17]:
# Remember variance is a measure of how spread out a data set is
# Get Netflix variance
mult_df['NFLX'].var()

# Annualize by getting the number of samples and multiply
#253 is the number of days the stock market is open.
days = len(mult_df.index) # 253

nflx_a_var = mult_df['NFLX'].var() * 253
nflx_a_var

0.21435236996843518

## Get Covariance of Stocks

Covariance is the measure of the relationship between 2 blocks of data. The covariance of a
stock to itself is the variance of that variable. 

In [18]:
mult_df.cov() * 253

,FB,AMZN,AAPL,NFLX,GOOG
FB,0.211498,0.121090,0.165004,0.122340,0.141870
AMZN,0.121090,0.148918,0.125888,0.121261,0.100904
AAPL,0.165004,0.125888,0.218483,0.120969,0.135044
NFLX,0.122340,0.121261,0.120969,0.214352,0.094118
GOOG,0.141870,0.100904,0.135044,0.094118,0.147431


## Why do We Care About Risk      

Most investors don't handle massive flucuations in stock prices well. What we want to
do at the very least is to make them aware of how dramatically their portfolios 
returns may be. We can then do our best to minimize risk by adding other stocks
that have returns that aren't as closely correlated.

## Calculating a Portfolios Variance

When calculating the variance of a portfolio we must define its weight, or how much of
the portfolio it makes up. If you add up the weight of all stocks you get a value of 1.

$ w_1, w_2 $ = Stock Weights

$ \sigma_1, \sigma_2 $ = Stock Standard Deviations

Portfolio Variance = $ (w_1\sigma_1 + w_2\sigma_2)^2 $

Since $ (a+b)^2 = a^2 + 2ab + b^2 $

Then the Portfolio Variance = $ w_1^2\sigma_1^2 + 2w_1\sigma_1w_2\sigma_2\rho_12 + w_2^2\sigma_2^2 $

In [37]:
# Let's create a portfolio made up of Facebook & Newmont Corporation
port_list = ["FB", "NEM"]
port_df = merge_df_by_column_name('daily_return',  '2020-1-1', '2020-12-31', *port_list)
port_df

# Generate a Correlation Matrix to see that they are not correlated
port_df.corr()

# Get prices at the beginning of Jan 2020 to calculate weight of both stocks
price_df = merge_df_by_column_name('Adj Close',  '2020-1-1', '2020-12-31', *["FB","NEM"])
price_df.head()

# I'll buy 1 share of FB at 209.78 & 5 of NEM for 41.74 for a total of 208.70
# Total Value = $418.48
fb_wt = 209.78 / 418.48
nem_wt = 208.70 / 418.48
fb_wt = .5012 # .5012
nem_wt = .4988 # .4987 and we'll change it to .4988 so they equal 1
wts = np.array([fb_wt, nem_wt])

# The portfolio variance is found from matrix multiplication
# We transpose the weights, multiply by the covarience and then the weight to get 
# the covariance of the portfolio
wts.T
port_var = np.dot(wts.T, np.dot(port_df.cov() * 253, wts))
print("Portfolio Var :", port_var)
print("FB Var :", port_df["FB"].var() * 253)
print("NEM Var :", port_df["NEM"].var() * 253)

File Doesn't Exist


TypeError: 'NoneType' object is not subscriptable